In [1]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter
from qiskit_ibm_provider import IBMProvider
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import PassManager, Layout
from qiskit.transpiler.passes import SetLayout, ApplyLayout, ALAPSchedule, ALAPScheduleAnalysis, BasisTranslator, ASAPSchedule

In [2]:
provider = IBMProvider()

In [3]:
ibm_backend_name = 'ibmq_guadalupe'
backend = provider.get_backend(ibm_backend_name)

In [4]:
qregs = QuantumRegister(backend.num_qubits)
delay_param = Parameter('t')

From backend props we can see that all single qubit operations take 160 dt. Let us make the delays multiples of this for convinience.

In [5]:
delays = [ 160, 320, 640, 1280, 2560, 5120, 10240 ]
single_qubit_gate_duration = 160

# Empty Circuit

In [6]:
empty_qc = QuantumCircuit(qregs, name='blank')
empty_qc.measure_all()

# Circuit Setup

In [7]:
def h_delay_h(middle_qc, qregs):
    qc = QuantumCircuit(qregs, name='H, delay, H')
    qc.h(qregs)
    qc = qc.compose(middle_qc, qregs)
    qc.h(qregs)
    qc.measure_all()
    return qc

def sx_delay_sx_x(middle_qc, qregs):
    qc = QuantumCircuit(qregs, name='sx, delay, sx, x')
    qc.sx(qregs)
    qc = qc.compose(middle_qc, qregs)
    qc.sx(qregs)
    qc.x(qregs)
    qc.measure_all()
    return qc

# Delay Circuits

In [8]:
middle_qc = QuantumCircuit(qregs, name='delay')
middle_qc.delay(delay_param, qregs)

In [9]:
h_delay_h_qc = h_delay_h(middle_qc, qregs)
sx_delay_sx_x_qc = sx_delay_sx_x(middle_qc, qregs)

In [10]:
h_delay_h_qcs, sx_delay_sx_x_qcs = [], []

for qc, qc_arr in zip((h_delay_h_qc, sx_delay_sx_x_qc), (h_delay_h_qcs, sx_delay_sx_x_qcs)):
    qc_arr.extend([qc.bind_parameters({delay_param: delay}) for delay in delays])


# Pairs of X gates

In [11]:
middle_qcs = []

for delay in delays:
    qc = QuantumCircuit(qregs, name='pair of X')
    n = delay//(2*single_qubit_gate_duration)
    for i in range(n):
        qc.x(qregs)
        qc.x(qregs)
    middle_qcs.append(qc)

In [12]:
h_xxxx_h_qcs, sx_xxxx_sx_x_qcs = [], []

for middle_qc in middle_qcs:
    h_delay_h_qc = h_delay_h(middle_qc, qregs)
    h_xxxx_h_qcs.append(h_delay_h_qc)
    sx_delay_sx_x_qc = sx_delay_sx_x(middle_qc, qregs)
    sx_xxxx_sx_x_qcs.append(sx_delay_sx_x_qc)

# XX with delays

In [13]:
single_qubit_gate_duration*3 + (single_qubit_gate_duration//2)*2

640

In [14]:
640*(2**4)

10240

In [15]:
delay_qc = QuantumCircuit(qregs)
delay_qc.delay(single_qubit_gate_duration//2)
delay_qc.x(qregs)
delay_qc.delay(single_qubit_gate_duration)
delay_qc.x(qregs)
delay_qc.delay(single_qubit_gate_duration//2)

In [16]:
middle_qcs = []
qc = QuantumCircuit(qregs, name='CP-DD-X')
qc = qc.compose(delay_qc, qregs)
middle_qcs.append(qc)

for i in range(4):
    qc = qc.compose(qc, qregs)
    middle_qcs.append(qc)

In [17]:
h_delayxx_h_qcs, sx_delayxx_sx_x_qcs = [], []

for middle_qc in middle_qcs:
    h_delay_h_qc = h_delay_h(middle_qc, qregs)
    h_delayxx_h_qcs.append(h_delay_h_qc)
    sx_delay_sx_x_qc = sx_delay_sx_x(middle_qc, qregs)
    sx_delayxx_sx_x_qcs.append(sx_delay_sx_x_qc)

# Collect all circuits and run on IBM Backend

In [18]:
all_circuits = [empty_qc, ]

experiment_qcs = [
    h_delay_h_qcs, sx_delay_sx_x_qcs,
    h_xxxx_h_qcs, sx_xxxx_sx_x_qcs,
    h_delayxx_h_qcs, sx_delayxx_sx_x_qcs
]

for qcs in experiment_qcs:
    all_circuits.extend(qcs)

In [19]:
len(all_circuits)

39

# Pickle Circuits

In [24]:
import pickle
pickle.dump( (empty_qc, experiment_qcs), open( "./other_experiments/delay_exps_3.p", "wb" ) )

## Run on Guadalupe

In [25]:
pm = generate_preset_pass_manager(0, backend)
tqcs = pm.run(all_circuits)

In [26]:
job = backend.run(tqcs, shots=8000)
job.update_tags(['delay_exps_3', ])
print('New job: ', job.job_id())

New job:  cidf12ru5mrpmc96e37g
